# NAIVE BAYES

In [1]:
#important libraries
import pandas as pd
import numpy as np
import nltk
import re
#importing stopwords is optional, in this case it decreased accuracy
#from nltk.corpus import stopwords
import itertools
import time




In [2]:
start_time = time.time()

import os
#os.chdir('/tmp/guest-pltjjp/Downloads')


data = pd.read_csv('TrumpvsBiden.csv')
data = data.iloc[:1000,:]
data.head()



,author,date,text
0,R0nnieBergman,2020-10-31 09:24:34,RT @standardnews: I created The Thick of It an...
1,Godhelpusameric,2020-10-31 09:24:34,RT @realDonaldTrump: “A Biden win would mean t...
2,Zehub,2020-10-31 09:24:34,VIDEO. Election présidentielle aux États-Unis ...
3,JamieParrigin,2020-10-31 09:24:34,RT @realDonaldTrump: President Obama used to s...
4,Powerfader54,2020-10-31 09:24:34,Tucker Carlson: Hunter Biden scandal shows med...


In [4]:
#stopset = set(stopwords.words('english'))

from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()

#comprehensive cleaning
def cleaning(text):
    txt = str(text)
    txt = re.sub(r"http\S+", "", txt)
    if len(txt) == 0:
        return 'no text'
    else:
        txt = txt.split()
        index = 0
        for j in range(len(txt)):
            if txt[j][0] == '@':
                index = j
        txt = np.delete(txt, index)
        if len(txt) == 0:
            return 'no text'
        else:
            words = txt[0]
            for k in range(len(txt)-1):
                words+= " " + txt[k+1]
            txt = words
            txt = re.sub(r'[^\w]', ' ', txt)
            if len(txt) == 0:
                return 'no text'
            else:
                txt = ''.join(''.join(s)[:2] for _, s in itertools.groupby(txt))
                txt = txt.replace("'", "")
                txt = nltk.tokenize.word_tokenize(txt)
                #data.content[i] = [w for w in data.content[i] if not w in stopset]
                for j in range(len(txt)):
                    txt[j] = lem.lemmatize(txt[j], "v")
                if len(txt) == 0:
                    return 'no text'
                else:
                    return txt
                



        


In [5]:
data['text'] = data['text'].map(lambda x: cleaning(x))
data.head()

,author,date,text
0,R0nnieBergman,2020-10-31 09:24:34,"[RT, standardnews, I, create, The, Thick, of, ..."
1,Godhelpusameric,2020-10-31 09:24:34,"[RT, A, Biden, win, would, mean, the, end, of,..."
2,Zehub,2020-10-31 09:24:34,"[Election, présidentielle, aux, États, Unis, c..."
3,JamieParrigin,2020-10-31 09:24:34,"[RT, President, Obama, use, to, say, that, if,..."
4,Powerfader54,2020-10-31 09:24:34,"[Carlson, Hunter, Biden, scandal, show, media,..."


In [6]:
data = data.reset_index(drop=True)
for i in range(len(data)):
    words = data.text[i][0]
    for j in range(len(data.text[i])-1):
        words+= ' ' + data.text[i][j+1]
    data.text[i] = words
    
x = int(np.round(len(data)*0.75))
train = data.iloc[:x,:].reset_index(drop = True)
test = data.iloc[x:,:].reset_index(drop = True)
    


In [7]:
from textblob.classifiers import NaiveBayesClassifier as NBC

training_corpus = []

for k in range(len(train)):
    training_corpus.append((train.text[k], train.text[k]))
    
test_corpus = []

for l in range(len(test)):
    test_corpus.append((test.text[l], test.text[l]))



In [8]:
model = NBC(training_corpus)

print(model.accuracy(test_corpus))



0.024


In [ ]:
from sklearn.metrics import classification_report

predictions = []
for m in range(len(test)):
    predictions.append(model.classify(test.text[m]))
print(classification_report(test, predictions))
    


In [ ]:
predictions_df = pd.DataFrame({'Content':test.text, 'Emotion_predicted':predictions, 'Emotion_actual':test.sentiment})
predictions_df.to_csv('naive_emotion_recognizer.csv', index = False)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")